In [2]:
# Test the code for the shapley function
import numpy as np
import openturns as ot

In [4]:
# inputs
mean = np.zeros(3)

cov = np.array([[1.0,0,0], [0,1,1.8], [0, 1.8, 4.0]])
cov = ot.CovarianceMatrix(cov)

#X = np.array([[0,0.0,0], [1,1,1]])

# inputs for Conditional mean function
#dependent_ind = np.array([1,0])
#given_ind= np.array([2])
#X_given= np.array([10])

In [5]:
# Definition of the toy model
def model(X):
    return np.sum(X,1)

In [6]:
# Conditional multivariate mean function

def condMVN(mean, cov, dependent_ind, given_ind, X_given):
    """ Returns conditional mean and variance of X[dependent.ind] | X[given.ind] = X.given
    where X is multivariateNormal(mean = mean, covariance = cov)"""
    
    cov = np.array(cov)
    
    B = cov.take(dependent_ind, axis = 1)
    B = B[dependent_ind]
    
    C = cov.take(dependent_ind, axis = 1)
    C = C[given_ind]
    
    D = cov.take(given_ind, axis = 1)
    D = D[given_ind]
    
    CDinv = np.dot(np.transpose(C),np.linalg.inv(D))
    
    condMean = mean[dependent_ind] + np.dot(CDinv,(X_given - mean[given_ind]))
    condVar = B - np.dot(CDinv,C)
    condVar = ot.CovarianceMatrix(condVar)
    
    return condMean,condVar

In [7]:
# Function to simulate conditional gausian distribution

def r_condMVN(n, mean, cov, dependent_ind, given_ind, X_given):
    """ Function to simulate conditional gaussian distribution of X[dependent.ind] | X[given.ind] = X.given
    where X is multivariateNormal(mean = mean, covariance = cov)"""
    
    cond_mean,cond_var = condMVN(mean, cov, dependent_ind, given_ind, X_given)
    distribution = ot.Normal(cond_mean,cond_var)
    return distribution.getSample(n)

In [9]:
# Definition of Shapely indices function

Nv = 10**4
No = 10**3
Ni = 3

Sj = np.array([0,1,2])
#Sjc= np.array([2])
xjc = np.array([10])
#xjc = X
n = Nv

d = 3
perms_tool = ot.KPermutations(d, d)
perms = perms_tool.generate()

m = perms.getSize() # number of permutations

def Xall(n):
    distribution = ot.Normal(mean,cov)
    return distribution.getSample(n)

def Xcond(n, Sj, Sjc, xjc):
    if Sjc is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(Sj, axis = 1)
        cov_int = cov_int[Sj]        
        cov_int = ot.CovarianceMatrix(cov_int)
        distribution = ot.Normal(mean[Sj],cov_int)
        return distribution.getSample(n)
    else:
        return r_condMVN(n,mean = mean, cov = cov, dependent_ind = Sj, given_ind = Sjc, X_given = xjc)

In [10]:
# example to calculate Xcond set of conditional inputs
# question: where do the inputs come from in respy?
cov_int = np.array(cov)
cov_int = cov_int.take(Sj, axis = 1)
cov_int = cov_int[Sj]        
cov_int = ot.CovarianceMatrix(cov_int)
distribution = ot.Normal(mean[Sj],cov_int)
distribution.getSample(n)

class=Sample name=Normal implementation=class=SampleImplementation name=Normal size=10000 dimension=3 description=[X0,X1,X2] data=[[0.608202,-1.26617,-2.66118],[1.20548,-2.18139,-3.62133],[-0.355007,1.43725,3.29377],...,[-1.65341,-0.125826,0.151116],[0.500173,-0.683907,-1.12957],[-0.363896,-0.594717,-0.212625]]

In [24]:
X = np.zeros((Nv+m*(d-1)*No*Ni, d)) 


In [25]:
X.shape

(46000, 3)

In [26]:
X[:Nv,:] = Xall(Nv)

In [29]:
X

array([[ 1.93359611, -1.16102681, -2.26262865],
       [-1.48112153,  1.0013619 ,  2.67215229],
       [ 0.51477535, -1.13043203, -2.07312641],
       ...,
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])